<a href="https://colab.research.google.com/github/mostafa-asefy/neural-networks-from-zero/blob/main/Artificial_Neural_Networks_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Class Definition

In [ ]:
class NeuralNetwork:

    # initialize the neural network 
    def __init__():
        pass

    # train the neural network
    def train():
        pass

    # query the neural network
    def query():
        pass